In [2]:

import os
import numpy as np
import zipfile
import pandas as pd
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import csv
import cv2

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="1Ew0Rg3kbVvfDLHUTBGf")
project = rf.workspace("yzu").project("good-bad-bean")
dataset = project.version(1).download("tensorflow")


                                              0.0/56.3 kB ? eta -:--:--
     --------------------                   30.7/56.3 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 56.3/56.3 kB 731.6 kB/s eta 0:00:00
                                              0.0/155.3 kB ? eta -:--:--
     ----------                              41.0/155.3 kB 1.9 MB/s eta 0:00:01
     ---------------                         61.4/155.3 kB 1.6 MB/s eta 0:00:01
     ---------------                         61.4/155.3 kB 1.6 MB/s eta 0:00:01
     -------------------                   81.9/155.3 kB 651.6 kB/s eta 0:00:01
     ---------------------                 92.2/155.3 kB 435.7 kB/s eta 0:00:01
     ---------------------                 92.2/155.3 kB 435.7 kB/s eta 0:00:01
     --------------------------           112.6/155.3 kB 385.0 kB/s eta 0:00:01
     ------------------------------------ 155.3/155.3 kB 440.8 kB/s eta 0:00:00
                                              0.0/178.7

Extracting Dataset Version Zip to Good-Bad-Bean-1 in tensorflow:: 100%|██████████| 4359/4359 [00:01<00:00, 2224.19it/s]


In [5]:
TRAINING_FILE = 'Good-Bad-Bean-1/train/_annotations.csv'

VALIDATION_FILE = 'Good-Bad-Bean-1/valid/_annotations.csv'

In [6]:
with open(TRAINING_FILE) as training_file:
  line = training_file.readline()
  print(f"First line (header) looks like this:\n{line}")
  line = training_file.readline()
  print(f"Each subsequent line (data points) look like this:\n{line}")

First line (header) looks like this:
filename,width,height,class,xmin,ymin,xmax,ymax

Each subsequent line (data points) look like this:
72a2b8f8-c7ca-4222-b2a1-9b82ac7990f4_45_jpg.rf.d17ad7cb4225bfc8720a0543682e6c22.jpg,416,416,good bean,56,1,341,397



In [7]:
def parse_data_from_input(filename):
    dataframe = pd.read_csv(filename)
    label_mapping = {'good bean': 0, 'bad bean': 1}
    dataframe['class'] = dataframe['class'].map(label_mapping)

    images = []
    labels = []

    for _, row in dataframe.iterrows():
        image_filename = row['filename']
        image_path = os.path.join(os.path.dirname(filename), image_filename)
        image = cv2.imread(image_path)
        image = cv2.blur(image, (5, 5), 0)

        x_min = row['xmin']
        y_min = row['ymin']
        x_max = row['xmax']
        y_max = row['ymax']

        cropped_image = image[y_min:y_max, x_min:x_max]
        resized_image = cv2.resize(cropped_image, (224, 224))
        reshaped_image = np.expand_dims(resized_image, axis=0)
        images.append(reshaped_image)
        labels.append(row['class'])

    images = np.concatenate(images, axis=0)
    labels = np.array(labels)

    return images, labels

In [8]:

# Test your function
training_images, training_labels = parse_data_from_input(TRAINING_FILE)
validation_images, validation_labels = parse_data_from_input(VALIDATION_FILE)

print(f"Training images has shape: {training_images.shape} and dtype: {training_images.dtype}")
print(f"Training labels has shape: {training_labels.shape} and dtype: {training_labels.dtype}")
print(f"Validation images has shape: {validation_images.shape} and dtype: {validation_images.dtype}")
print(f"Validation labels has shape: {validation_labels.shape} and dtype: {validation_labels.dtype}")

Training images has shape: (3807, 224, 224, 3) and dtype: uint8
Training labels has shape: (3807,) and dtype: int32
Validation images has shape: (363, 224, 224, 3) and dtype: uint8
Validation labels has shape: (363,) and dtype: int32


In [9]:
def train_val_generators(training_images, training_labels, validation_images, validation_labels):
    """
    Creates the training and validation data generators

    Args:
        training_images (array): parsed images from the train CSV file
        training_labels (array): parsed labels from the train CSV file
        validation_images (array): parsed images from the test CSV file
        validation_labels (array): parsed labels from the test CSV file

    Returns:
        train_generator, validation_generator - tuple containing the generators
    """


    # Instantiate the ImageDataGenerator class
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Pass in the appropriate arguments to the flow method
    train_generator = train_datagen.flow(
        x=training_images,
        y=training_labels,
        batch_size=32
    )

    # Instantiate the ImageDataGenerator class (don't forget to set the rescale argument)
    # Remember that validation data should not be augmented
    validation_datagen = ImageDataGenerator(rescale=1./255)

    # Pass in the appropriate arguments to the flow method
    validation_generator = validation_datagen.flow(
        x=validation_images,
        y=validation_labels,
        batch_size=32
    )

    return train_generator, validation_generator

train_generator, validation_generator = train_val_generators(training_images, training_labels, validation_images, validation_labels)

print(f"Images of training generator have shape: {train_generator[0][0].shape}")
print(f"Labels of training generator have shape: {train_generator[0][1].shape}")
print(f"Images of validation generator have shape: {validation_generator[0][0].shape}")
print(f"Labels of validation generator have shape: {validation_generator[0][1].shape}")

Images of training generator have shape: (32, 224, 224, 3)
Labels of training generator have shape: (32,)
Images of validation generator have shape: (32, 224, 224, 3)
Labels of validation generator have shape: (32,)


In [18]:
def create_model():
    # load the InceptionResNetV2 architecture with imagenet weights as base
    base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
    base_model.trainable=False     
    # Define the model
    model = tf.keras.Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [16]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') is not None and logs.get('accuracy') > 0.93):
            print("\nReached target accuracy so cancelling training!")
            self.model.stop_training = True
        

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
model = create_model()
#Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Model checkpoint callback
checkpoint = ModelCheckpoint('best_model_coba3.h5', monitor='val_loss', save_best_only=True)

# Learning rate scheduling callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)

#accuracy threshold callback
callbacks = myCallback()

#train the model
model.fit(train_generator,
          epochs=100,
          validation_data=validation_generator,
          callbacks=[callbacks,early_stopping, checkpoint, reduce_lr])

Epoch 1/100
119/119 [==============================] - ETA: 0s - loss: 0.6399 - accuracy: 0.7460

RuntimeError: Can't decrement id ref count (unable to extend file properly)

In [ ]:
# Save the entire model as a SavedModel.
model.save('crop_classification_model2')


INFO:tensorflow:Assets written to: crop_classification_model2\assets


INFO:tensorflow:Assets written to: crop_classification_model2\assets


In [24]:
#save the model as hdf5 file
model.save('crop_classification_model4.h5')

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing.image import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
    # Load and preprocess the image
    path = '/content/' + fn
    img = load_img(path, target_size=(64, 64))
    x = img_to_array(img)
    x /= 255.0
    x = np.expand_dims(x, axis=0)

    # Make predictions
    predictions = model.predict(x)
    predicted_label = int(round(predictions[0][0]))

    # Print the predicted label
    if predicted_label == 0:
        print(f"{fn} is a good bean")
    else:
        print(f"{fn} is a bad bean")

ModuleNotFoundError: No module named 'google.colab'